In [1]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from astroquery.mpc import MPC

# Cobs API

In [2]:
# Verificar la conexión a internet.
def verificar_conexion():
    try:
        requests.get("http://www.google.com", timeout=5)
        print('✅ Conectado a internet.')
        return True
    
    except requests.ConnectionError:
        print('🛑 Sin conexión a internet.')
        return False

In [3]:
# Conexión con la API de COBS
nombre_cometa =  'C/2023 A3'
Link_cops_API = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date=&to_date=&exclude_faint=False&exclude_not_accurate=False'

if verificar_conexion():
    response = requests.get(Link_cops_API)


if response.status_code == 200:
    content = response.json()
    print('✅ Base de datos actualizada.')

else:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Base de datos actualizada.


In [4]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content['objects'])
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1281 entries, 0 to 1280
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       1281 non-null   object 
 1   obs_date                   1281 non-null   object 
 2   comet                      1281 non-null   object 
 3   observer                   1281 non-null   object 
 4   location                   596 non-null    object 
 5   extinction                 111 non-null    object 
 6   obs_method                 1281 non-null   object 
 7   comet_visibility           6 non-null      object 
 8   magnitude                  1281 non-null   object 
 9   conditions                 29 non-null     object 
 10  ref_catalog                1281 non-null   object 
 11  instrument_aperture        1281 non-null   object 
 12  instrument_type            1281 non-null   object 
 13  instrument_focal_ratio     1062 non-null   float

In [5]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 1281
Variables: 47


In [6]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
256,C,2024-06-26 21:50:24,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Pieter-Jan', 'last_name': 'Dek...",None,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,10.2,None,...,0.04,None,1.1,1.1,s,,None,NaN,,2024-06-27 08:32:00
38,V,2024-09-26 12:14:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Carl', 'last_name': 'Hergenrot...","Tanque Verde, Arizona",a,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,3.0,None,...,None,None,None,None,None,,None,NaN,,2024-09-26 17:33:48
692,V,2024-05-04 21:50:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Eryk', 'last_name': 'Banach', ...",None,None,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,10.3,None,...,None,None,None,None,None,,21.1,3.0,,2024-05-05 00:03:18
478,C,2024-05-25 12:00:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Katsumi', 'last_name': 'Yoshim...",None,None,"{'key': 'D', 'name': 'Johnson/Bessel/Kron/Cous...",None,11.7,None,...,None,13.00,2.0,2.0,s,,None,NaN,,2024-05-31 01:19:57
1131,C,2023-09-06 19:40:48,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Pieter-Jan', 'last_name': 'Dek...",None,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,15.3,None,...,0.14,None,1.1,1.1,s,,None,NaN,,2023-09-07 20:24:30


In [7]:
# Métodos de observación
cometa_df.obs_method.apply(lambda registro: f'{registro['key']}: {registro['name']}').value_counts()

obs_method
C: Unfiltered total CCD magnitude                                                   345
Z: CCD Visual equivalent                                                            294
M: Modified-Out method                                                              225
S: In-Out method                                                                    211
V: Johnson/Bessel/Kron/Cousins V with CCD                                            89
k: Kron/Cousins R with CCD                                                           30
B: Simple Out-Out method                                                             22
D: Johnson/Bessel/Kron/Cousins B with CCD                                            18
H: Kron/Cousins I with CCD                                                           17
I: In-focus                                                                          16
c: Unfiltered nuclear CCD magnitude                                                   8
d: Astrodon G filter 

In [8]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'])
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [9]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1281 entries, 0 to 1280
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1281 non-null   object        
 1   obs_date        1281 non-null   datetime64[ns]
 2   magnitude       1281 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 30.2+ KB


In [10]:
# Con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 1281
Variables: 3


In [11]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
1174,C,2023-06-12,17.3
403,M,2024-06-01,10.4
830,S,2024-04-26,10.4
587,M,2024-05-11,10.1
700,M,2024-05-04,10.6


In [12]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# MPC API usando astroquery.

In [13]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
fechas = (fecha_final - fecha_inicial).days + 1 if (fecha_final - fecha_inicial).days <= 1441 else 1441

ephemeris = MPC.get_ephemeris(nombre_cometa, start = str(fecha_inicial), number = fechas) # type: ignore

ephemeris_df = ephemeris.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower().str.replace(' ', '_')
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,2023-02-25,230.565833,-1.256389,6.946,7.290,106.6,7.5,17.6,8.17,318.9
1,2023-02-26,230.528333,-1.214722,6.921,7.281,107.6,7.4,17.6,8.51,317.4
2,2023-02-27,230.488750,-1.172778,6.896,7.272,108.6,7.4,17.6,8.86,316.0
3,2023-02-28,230.446667,-1.130000,6.871,7.264,109.6,7.4,17.6,9.21,314.7
4,2023-03-01,230.401667,-1.086389,6.847,7.255,110.6,7.3,17.6,9.56,313.4
...,...,...,...,...,...,...,...,...,...,...
579,2024-09-26,160.411250,-6.112222,0.984,0.394,22.9,81.2,3.2,90.36,89.1
580,2024-09-27,161.098750,-6.091389,0.940,0.392,23.0,87.4,3.1,115.48,87.6
581,2024-09-28,161.967500,-6.046667,0.897,0.391,22.9,93.8,3.0,144.73,86.6
582,2024-09-29,163.045833,-5.976111,0.853,0.393,22.7,100.5,2.9,178.43,85.9


In [14]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           584 non-null    datetime64[ns]
 1   ra             584 non-null    float64       
 2   dec            584 non-null    float64       
 3   delta          584 non-null    float64       
 4   r              584 non-null    float64       
 5   elongation     584 non-null    float64       
 6   phase          584 non-null    float64       
 7   v              584 non-null    float64       
 8   proper_motion  584 non-null    float64       
 9   direction      584 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 45.8 KB


In [15]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [16]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,2023-02-25,6.946,7.290,7.5
1,2023-02-26,6.921,7.281,7.4
2,2023-02-27,6.896,7.272,7.4
3,2023-02-28,6.871,7.264,7.4
4,2023-03-01,6.847,7.255,7.3
...,...,...,...,...
579,2024-09-26,0.984,0.394,81.2
580,2024-09-27,0.940,0.392,87.4
581,2024-09-28,0.897,0.391,93.8
582,2024-09-29,0.853,0.393,100.5


# Unión de las bases de datos.

In [17]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,M,2024-09-30,2.2,0.810,0.396,107.3
1,I,2024-09-30,2.0,0.810,0.396,107.3
2,I,2024-09-30,2.4,0.810,0.396,107.3
3,M,2024-09-30,2.3,0.810,0.396,107.3
4,t,2024-09-30,1.8,0.810,0.396,107.3
...,...,...,...,...,...,...
1276,C,2023-02-27,18.0,6.896,7.272,7.4
1277,C,2023-02-26,18.1,6.921,7.281,7.4
1278,C,2023-02-25,18.2,6.946,7.290,7.5
1279,C,2023-02-25,17.9,6.946,7.290,7.5


In [18]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1281 entries, 0 to 1280
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1281 non-null   object        
 1   obs_date        1281 non-null   datetime64[ns]
 2   magnitude       1281 non-null   float64       
 3   delta           1281 non-null   float64       
 4   r               1281 non-null   float64       
 5   phase           1281 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 60.2+ KB


In [19]:
# Reducción de la magnitud aparente
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida
0,M,2024-09-30,2.2,0.810,0.396,107.3,4.669099
1,I,2024-09-30,2.0,0.810,0.396,107.3,4.469099
2,I,2024-09-30,2.4,0.810,0.396,107.3,4.869099
3,M,2024-09-30,2.3,0.810,0.396,107.3,4.769099
4,t,2024-09-30,1.8,0.810,0.396,107.3,4.269099
...,...,...,...,...,...,...,...
1276,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744
1277,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201
1278,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688
1279,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688


In [20]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna

In [71]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 9

curva_de_luz_promediada_df = curva_de_luz_procesada_df.copy()
curva_de_luz_promediada_df['promedio_movil'] = curva_de_luz_promediada_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_promediada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,M,2024-09-30,2.2,0.810,0.396,107.3,4.669099,NaN
1,I,2024-09-30,2.0,0.810,0.396,107.3,4.469099,NaN
2,I,2024-09-30,2.4,0.810,0.396,107.3,4.869099,NaN
3,M,2024-09-30,2.3,0.810,0.396,107.3,4.769099,NaN
4,t,2024-09-30,1.8,0.810,0.396,107.3,4.269099,NaN
...,...,...,...,...,...,...,...,...
1276,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744,9.492568
1277,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201,9.506367
1278,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688,9.530109
1279,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688,9.512041


In [72]:
# Curva de luz Promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='obs_date', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [73]:
# Creación del data frame curva de luz agrupada
curva_de_luz_interna_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').max()
curva_de_luz_interna_df = curva_de_luz_interna_df.reset_index()
curva_de_luz_interna_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,9.530109
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,9.506367
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,9.523162
3,2023-02-28,C,18.2,6.871,7.264,7.4,9.709021,9.531535
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,9.528274
...,...,...,...,...,...,...,...,...
314,2024-09-26,k,4.4,0.984,0.394,81.2,6.457543,5.424210
315,2024-09-27,k,4.2,0.940,0.392,87.4,6.367930,4.994968
316,2024-09-28,k,3.5,0.897,0.391,93.8,5.775154,4.941821
317,2024-09-29,i,2.7,0.853,0.393,100.5,5.073292,4.628382


In [74]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente)

In [75]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_promediada_df = curva_de_luz_procesada_df.copy()
curva_de_luz_promediada_df['promedio_movil'] = curva_de_luz_promediada_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_promediada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,M,2024-09-30,2.2,0.810,0.396,107.3,4.669099,NaN
1,I,2024-09-30,2.0,0.810,0.396,107.3,4.469099,NaN
2,I,2024-09-30,2.4,0.810,0.396,107.3,4.869099,NaN
3,M,2024-09-30,2.3,0.810,0.396,107.3,4.769099,NaN
4,t,2024-09-30,1.8,0.810,0.396,107.3,4.269099,NaN
...,...,...,...,...,...,...,...,...
1276,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744,9.500728
1277,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201,9.507570
1278,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688,9.500104
1279,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688,9.494158


In [81]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='obs_date', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Max Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [76]:
# Creación del data frame curva de luz agrupada
curva_de_luz_externa_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').min()
curva_de_luz_externa_df = curva_de_luz_externa_df.reset_index()
curva_de_luz_externa_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,9.494158
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,9.507570
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,9.495765
3,2023-02-28,C,17.7,6.871,7.264,7.4,9.209021,9.505089
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,9.552253
...,...,...,...,...,...,...,...,...
314,2024-09-26,C,2.6,0.984,0.394,81.2,4.657543,5.054225
315,2024-09-27,D,2.5,0.940,0.392,87.4,4.667930,4.788408
316,2024-09-28,B,1.9,0.897,0.391,93.8,4.175154,4.631765
317,2024-09-29,I,1.5,0.853,0.393,100.5,3.873292,4.485182


In [82]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_df, x=curva_de_luz_externa_df.obs_date, y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Comparación de las curvas de luz

In [79]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_df.obs_date, y=curva_de_luz_externa_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_df.obs_date, y=curva_de_luz_interna_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Promedio Movil')
fig.show()